<a href="https://colab.research.google.com/github/hedgreneric/LunarLander/blob/main/LunarLander_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and Importing Dependices

In [ ]:
!pip install tensorflow==1.15.0 tensorflow-gpu==1.15.0 stable_baselines gym box2d-py --user

#for display
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gym 
from stable_baselines import ACER
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.evaluation import evaluate_policy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/root/.local/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [ ]:
#for display
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""


def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

In [ ]:
environment_name = 'LunarLander-v2'

# Test Random Environment

In [ ]:
env = gym.make(environment_name)

In [ ]:
episodes = 15
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-337.6699622412433
Episode:2 Score:-248.82866450926332
Episode:3 Score:-325.53680241296
Episode:4 Score:-176.39678102613362
Episode:5 Score:-257.60737653776926
Episode:6 Score:-86.37673626947227
Episode:7 Score:-240.59746437687832
Episode:8 Score:-98.08478963338996
Episode:9 Score:-236.17935762031337
Episode:10 Score:-211.7596118951949
Episode:11 Score:-184.8217477837233
Episode:12 Score:-220.06888725569894
Episode:13 Score:-318.4491384077048
Episode:14 Score:-283.57033408921245
Episode:15 Score:-83.41234986419053


# Build and Train the Model

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = ACER('MlpPolicy', env, verbose = 1)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [ ]:
model.learn(total_timesteps=100000)

----------------------------------
| avg_norm_adj        | 3.9      |
| avg_norm_g          | 16.9     |
| avg_norm_grads_f    | 14.7     |
| avg_norm_k          | 2        |
| avg_norm_k_dot_g    | 16.9     |
| entropy             | 29.1     |
| explained_variance  | 0.000475 |
| fps                 | 0        |
| loss                | -0.476   |
| loss_bc             | -0       |
| loss_f              | -5.61    |
| loss_policy         | -5.61    |
| loss_q              | 10.9     |
| mean_episode_length | 0        |
| mean_episode_reward | 0        |
| norm_grads          | 3.45     |
| norm_grads_policy   | 2        |
| norm_grads_q        | 2.82     |
| total_timesteps     | 20       |
----------------------------------
----------------------------------
| avg_norm_adj        | 5.88     |
| avg_norm_g          | 25       |
| avg_norm_grads_f    | 21.7     |
| avg_norm_k          | 2        |
| avg_norm_k_dot_g    | 24.9     |
| entropy             | 29       |
| explained_variance

# Save and Test the Model

In [ ]:
evaluate_policy(model, env, n_eval_episodes=15, render=True)
env.close()

In [ ]:
model.save("ACER_model")

In [ ]:
del model

In [ ]:
model = ACER.load("ACER_model", env=env)

In [ ]:
env = wrap_env(gym.make(environment_name))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

    if done:
      break

In [ ]:
env.close()
show_video()